## Introduction

**Customer value** or **Customer Lifetime Value (CLTV)** is the total monetary value of transactions/purchases made by a customer with your business over his entire lifetime. Here the lifetime means the time period till your customer purchases with you before moving to your competitors.

## Useful Information & Metrics

* CLTV = (Customer_Value / Churn_Rate) x Profit_margin

* Customer_Value = Average_Order_Value * Purchase_Frequency

* Average_Order_Value = Total_Revenue / Total_Number_of_Orders

* Purchase_Frequency = Total_Number_of_Orders / Total_Number_of_Customers

* Churn_Rate = 1 - Repeat_Rate

* Profit_margin

### Loading Dataset & Understanding Data

In [4]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [7]:
# Read the data frame

df_ = pd.read_excel('../Customer Lifetime Value Prediction/online_retail_II.xlsx',
                       sheet_name='Year 2010-2011')
df= df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [9]:
df = df[df["Country"] == "United Kingdom"]
#df["Country"].nunique()
df.describe().T
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

In [10]:
def outlier_thresholds(dataframe, variable): # suppress outliers
    quartile1 = dataframe[variable].quantile(0.01) # We do a small filing of 0.01
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit # Equalize the values below the low limit to the low limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit # Equalize values above the up limit to the up limit

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [11]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

today_date = dt.datetime(2011, 12, 11)



cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})


In [12]:
cltv_df.columns = cltv_df.columns.droplevel(0)
#çıkan lambda dan dolayı drop ettik o kısmı sadece columns lar olsun diye,altta da tanımladık.
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
#alışveriş başına ne kadar harcama yapılmış buluyoruz
cltv_df.head()
cltv_df = cltv_df[cltv_df["monetary"] > 0]

cltv_df["recency"] = cltv_df["recency"] / 7
#haftalık için

cltv_df["T"] = cltv_df["T"] / 7

cltv_df = cltv_df[(cltv_df['frequency'] > 1)] #why should frequency be greater than 1
# Our aim is to analyze more than one customer, not one time visitor.
#An accurate prediction will not be made for the customer who comes #1 times, 
#so there is a possibility that it will affect the model badly, so we prefer not to include it.
# model's historical data is not enough for us


In [13]:
cltv_df

,recency,T,frequency,monetary
Customer ID,,,,
12747.0,52.285714,52.857143,11,381.455455
12748.0,53.142857,53.428571,209,154.564163
12749.0,29.857143,30.571429,5,814.488000
12820.0,46.142857,46.714286,4,235.585000
12822.0,2.285714,12.571429,2,474.440000
...,...,...,...,...
18272.0,34.857143,35.285714,6,513.096667
18273.0,36.428571,36.857143,3,68.000000
18282.0,16.857143,18.142857,2,89.025000


## Task - 6 months CLTV Prediction

In [14]:
# 2. BG-NBD Model

bgf = BetaGeoFitter(penalizer_coef=0.001) #used to prevent overfitting
#It is referred to as the penalty coefficient, it can lead to reasons that may be over-learning in small data sets,


bgf.fit(cltv_df['frequency'], cltv_df['recency'], cltv_df['T'])

ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

cltv_df.sort_values("expected_average_profit", ascending=False).head(20)
cltv_df.sort_values("expected_average_profit", ascending=False).head()


,recency,T,frequency,monetary,expected_average_profit
Customer ID,,,,,
14088.0,44.571429,46.142857,13,3859.601538,3911.319974
18102.0,52.285714,52.571429,60,3584.887750,3595.192779
14096.0,13.857143,14.571429,17,3159.077059,3191.387434
17511.0,52.857143,53.428571,31,2921.951935,2938.274839
15749.0,13.857143,47.571429,3,2521.446667,2675.122485


In [29]:
# Calculation of CLTV with BG-NBD and GG model.
##############################################################

cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 months
                                   freq="W",  # Frequency of T.
                                   discount_rate=0.01)

cltv.head(15)

Customer ID
12747.0     1937.009093
12748.0    12366.071920
12749.0     3445.922970
12820.0      631.934113
12822.0     1612.132952
12823.0     1095.502620
12826.0      757.003898
12827.0     1009.062931
12828.0     1125.060613
12829.0       16.115941
12830.0     4662.975539
12832.0      765.780228
12836.0     1749.826288
12838.0      571.744409
12839.0     2495.218132
Name: clv, dtype: float64

## Task 2 - CLTV analysis consisting of different time periods

In [15]:
cltv1 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1,  # 1 month
                                   freq="W",  # Frequency of T.
                                   discount_rate=0.01)

rfm_cltv1_final = cltv_df.merge(cltv1,how="left", on="Customer ID")
rfm_cltv1_final.sort_values(by="clv", ascending=False).head()


,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
18102.0,52.285714,52.571429,60,3584.887750,3595.192779,14884.500498
14096.0,13.857143,14.571429,17,3159.077059,3191.387434,9855.142320
17450.0,51.285714,52.571429,46,2629.529891,2639.419554,8434.507606
17511.0,52.857143,53.428571,31,2921.951935,2938.274839,6394.139434
16684.0,50.428571,51.285714,28,2120.046964,2133.203870,4360.925582


In [17]:
cltv12 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=12,  # 12 month
                                   freq="W",  # Frequency of T.
                                   discount_rate=0.01)

rfm_cltv12_final = cltv_df.merge(cltv12, on="Customer ID", how="left")
rfm_cltv12_final.head()

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
12747.0,52.285714,52.857143,11,381.455455,387.822977,3698.318488
12748.0,53.142857,53.428571,209,154.564163,154.708635,23624.576677
12749.0,29.857143,30.571429,5,814.488000,844.095338,6538.680790
12820.0,46.142857,46.714286,4,235.585000,247.081182,1204.318808
12822.0,2.285714,12.571429,2,474.440000,520.829235,3029.875461


In [18]:
# 2. Analyze the 10 highest individuals at 1 month CLTV and the 10 highest at 12 months. Is there a difference?
# If there is, why do you think it might be?
rfm_cltv1_final.sort_values("clv", ascending=False).head(15)
rfm_cltv12_final.sort_values("clv", ascending=False).head(15)

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
18102.0,52.285714,52.571429,60,3584.887750,3595.192779,163586.717976
14096.0,13.857143,14.571429,17,3159.077059,3191.387434,104893.741912
17450.0,51.285714,52.571429,46,2629.529891,2639.419554,92691.902873
17511.0,52.857143,53.428571,31,2921.951935,2938.274839,70283.954919
16684.0,50.428571,51.285714,28,2120.046964,2133.203870,47889.189123
13694.0,52.714286,53.428571,50,1267.362600,1271.785498,47870.662086
14088.0,44.571429,46.142857,13,3859.601538,3911.319974,47687.833143
15311.0,53.285714,53.428571,91,667.596813,668.894487,45066.570490
13089.0,52.285714,52.857143,97,605.186598,606.293686,43794.263478


In [22]:
cltv.shape
cltv = cltv.reset_index()
cltv.sort_values(by="clv", ascending=False).head(50)
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head()
cltv_final.sort_values(by="clv", ascending=False)[10:30]
cltv_final["cltv_segment"] = pd.qcut(cltv_final["clv"], 4, labels=["Premium", "Gold", "Silver", "Bronze"])

In [24]:
cltv_final.head(15)

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv,cltv_segment
0,12747.0,52.285714,52.857143,11,381.455455,387.822977,1937.009093,Bronze
1,12748.0,53.142857,53.428571,209,154.564163,154.708635,12366.071920,Bronze
2,12749.0,29.857143,30.571429,5,814.488000,844.095338,3445.922970,Bronze
3,12820.0,46.142857,46.714286,4,235.585000,247.081182,631.934113,Gold
4,12822.0,2.285714,12.571429,2,474.440000,520.829235,1612.132952,Silver
5,12823.0,31.571429,42.428571,5,351.900000,365.132831,1095.502620,Silver
6,12826.0,51.714286,52.285714,7,210.674286,216.496671,757.003898,Gold
7,12827.0,5.428571,6.428571,3,143.383333,153.372119,1009.062931,Silver
8,12828.0,18.142857,18.714286,6,169.785000,175.407161,1125.060613,Silver
9,12829.0,3.285714,51.571429,2,138.110000,153.067110,16.115941,Premium


In [25]:
# Does it make sense to divide customers into 4 groups based on CLTV scores?
cltv_final.groupby("cltv_segment").agg({"mean"})

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv
,mean,mean,mean,mean,mean,mean,mean
cltv_segment,,,,,,,
Premium,15705.926905,22.050433,40.496778,3.065319,177.497783,192.305958,269.482090
Gold,15521.657321,30.853138,38.177793,3.992212,260.682948,278.057628,710.892256
Silver,15596.336449,29.838896,35.117045,5.454829,351.986856,370.495976,1271.330816
Bronze,15390.838258,31.460564,34.519218,11.290824,586.303358,608.698256,3806.227878


## Task 3 - Segmentation and Action Recommendations

### Silver and Bronze Segmentation Actions

**Packed Content That Keeps Customers Engaged**

- If you provide accountancy services, send a weekly or monthly email telling clients how much money you’ve helped them save that month.
- If you provide help desk software, send a monthly email to clients informing them how many support tickets they successfully solved.
- If you sell an environmentally friendly product, email your customers to tell them how much less carbon dioxide they’ve produced by using a competing product.

**Collect Actionable Feedback**
- You should definitely collect and store all the feedback you receive in one place and share it across all departments. Also, have a team that’s responsible for monitoring customer sentiment online (on social media, communication channels, review websites, etc.).
- Send out customer satisfaction surveys 

**Upsell and Cross-Sell**

- **Offer bundle products**: Group different products and services together and sell them at a lower price than what they would have been sold for separately. It helps you increase the value of customer purchases by offering complementary products in a combo.
- **Add complementary products at checkout**: Products you offer as cross-sells should address the needs of your customer. Ideally, they should complement the primary product in the shopping cart.

- **Free shipping with minimum spend**: The word “free” by itself excites many buyers, but when you combine it with free shipping with a minimum order size condition, it makes it even more appealing. Customers are more likely to spend more money – that they would have otherwise spent on shipping – to buy something else.